In [89]:
import pandas as pd
from pandas.io.json import build_table_schema
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import missingno as msno
import numpy as np
import scipy as sp
import os
import datetime
#import xgboost as xgb
# import visualization as vis
from pandas.plotting import scatter_matrix
import warnings
%matplotlib inline

In [90]:
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')


In [91]:
#drop training rows where full_sq is missing
train = train[~train['full_sq'].isnull()]

In [92]:
#impute 5 missing full_sq in test data
test.loc[test['id'] == 30938, 'full_sq'] = 37.80
test.loc[test['id'] == 35857, 'full_sq'] = 42.07
test.loc[test['id'] == 34670, 'full_sq'] = 122.60

#np.nanmedian(test.full_sq)

test.loc[test['id'] == 36824, 'full_sq'] = 50.42
test.loc[test['id'] == 35108, 'full_sq'] = 50.42

In [93]:
#create prediction matrix
y_pred = pd.DataFrame({'id': test.id.values, 'price_doc': [0]*test.shape[0]})
print y_pred.shape
y_pred.head()


(7662, 2)


,id,price_doc
0,30474,0
1,30475,0
2,30476,0
3,30477,0
4,30478,0


In [79]:
nbhd = 'Tverskoe'
test[test['sub_area']==nbhd][['full_sq', 'id']].sort_values('full_sq')

,full_sq,id
5604,22.50,36078
3847,24.90,34321
924,25.70,31398
4598,25.80,35072
979,29.50,31453
617,30.15,31091
1404,30.41,31878
2796,30.90,33270
3144,31.10,33618
1046,32.10,31520


In [101]:
from scipy import stats
from sklearn import linear_model
ols = linear_model.LinearRegression()

y_pred = test[['id']]

for nbhd in set(train['sub_area']):
    
    
    X = train[train['sub_area']==nbhd][['full_sq']]
    y = np.log(train[train['sub_area']==nbhd][['price_doc']]+1)
    ols.fit(X,y)
    
    nbhd_test = test[test['sub_area']==nbhd][['full_sq', 'id']]
    
    if nbhd_test.shape[0] < 1:
        continue
    
    nbhd_test['predict'] = np.exp(ols.predict(nbhd_test[['full_sq']]))-1
    
    y_pred = pd.merge(left=y_pred, right=nbhd_test[['predict', 'id']], how='outer', on='id')
    
    print ols.coef_, ols.intercept_
    if ols.coef_[0] <= 0:
        print nbhd
    
y_pred.head()

    

[[ 0.02272872]] [ 14.5451713]
[[ 0.01588318]] [ 14.78889189]
[[ 0.01652705]] [ 14.81033432]
[[ 0.01507643]] [ 14.69588127]
[[ 0.01152994]] [ 15.19411831]
[[ 0.0129737]] [ 14.99515329]
[[ 0.01878446]] [ 14.68024394]
[[ 0.01802225]] [ 15.16956633]
[[ 0.01950666]] [ 15.00734868]
[[ 0.01252846]] [ 15.23713515]
[[ 0.01374411]] [ 15.20728565]
[[ 0.00994405]] [ 14.73834947]
[[ 0.01591258]] [ 15.46012417]
[[ 0.02171026]] [ 14.46886736]
[[ 0.00695469]] [ 14.7405428]
[[ 0.00698167]] [ 14.89563906]
[[ 0.01545126]] [ 14.64150035]
[[ 0.0113507]] [ 15.30604993]
[[ 0.01029944]] [ 14.75736358]
[[ 0.01683185]] [ 14.7127481]
[[ 0.0156229]] [ 14.7450416]
[[ 0.0102489]] [ 15.0606314]
[[ 0.01715432]] [ 14.89568398]
[[ 0.0159062]] [ 14.76934651]
[[ 0.01318382]] [ 14.78359893]
[[ 0.01693405]] [ 14.72184854]
[[-0.02547123]] [ 16.54428194]
Poselenie Shhapovskoe
[[ 0.01680851]] [ 14.94714427]
[[ 0.01095064]] [ 15.2366795]
[[ 0.01730209]] [ 14.99269091]
[[ 0.00786846]] [ 14.64163428]
[[ 0.01256977]] [ 14.7644859

,id,predict_x,predict_y,predict_x,predict_y,predict_x,predict_y,predict_x,predict_y,predict_x,...,predict_y,predict_x,predict_y,predict_x,predict_y,predict_x,predict_y,predict_x,predict_y,predict
0,30474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
y_pred = pd.DataFrame({'id': y_pred.id.values, 'price_doc':y_pred.iloc[:, 1:].sum(axis=1)})

y_pred.head()

,id,price_doc
0,30474,4.794935e+06
1,30475,8.198590e+06
2,30476,4.910674e+06
3,30477,5.666985e+06
4,30478,4.710821e+06


In [96]:
y_pred.to_csv('linreg_by_subarea_pred.csv', index=False)

In [97]:
print np.nanmedian(y_pred['price_doc'])
print np.mean(y_pred['price_doc'])

5866099.72797
6662981.50233
